In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import davies_bouldin_score, normalized_mutual_info_score, silhouette_score
from sklearn.decomposition import PCA
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean
from sentence_transformers import SentenceTransformer
from scipy import stats
from scipy.stats import norm
import numpy as np
import pandas as pd
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import davies_bouldin_score, normalized_mutual_info_score, silhouette_score
from sklearn.decomposition import PCA
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean
from sentence_transformers import SentenceTransformer

In [ ]:
def cluster_sampling(kmeans, label, reduced_fea, cluster_idx, num_samples):
    incluster_dist_list = []
    cluster_center = kmeans.cluster_centers_[label]
    for idx in cluster_idx:
        point = reduced_fea[idx]
        dist = euclidean(point, cluster_center)
        incluster_dist_list.append(dist)
    normalizd_dist = np.array(incluster_dist_list)
    inverse_norm_dist = 1/normalizd_dist
    sampled_idx = np.random.choice(cluster_idx, num_samples, p=inverse_norm_dist/inverse_norm_dist.sum()) 
    
    return sampled_idx, inverse_norm_dist/inverse_norm_dist.sum()

def max_min_scaling(inputs):
    return (inputs  - min(inputs)) /(max(inputs) - min(inputs))

def in_cluster_sample(sent_list, sets, kmeans, kmean_cluster, sample_per_cluster, reduced_fea):
    sample_str = ''
    p_str = ''
    cluster_idx_str = ''
    sample_list = []
    prob_list = []
    annotate_sent_list = []
    un_annotate_sent_list = []
    cluster_idx_list = []
    
    for i in range(kmean_cluster):
        cluster_idx = np.where(kmeans.labels_ == i)[0].tolist() 
        # cluster_idx = np.where(kmeans.labels_ == 0)[0].tolist() 
        # sample = np.random.choice(cluster_idx, sample_per_cluster) 
        sample, p = cluster_sampling(kmeans, i, reduced_fea, cluster_idx, sample_per_cluster)
        diff_idx = list(set(cluster_idx) - set(sample))
        annotate_sent = []
        un_annotate_sent = []
        for j in sample:
            annotate_sent.append(sent_list[j])
            
        for z in diff_idx:
            un_annotate_sent.append(sent_list[z])
            
        sample_list += list(sample)
        prob_list.append(';'.join([str(item) for item in list(p)]))
        annotate_sent_list += list(annotate_sent) 
        un_annotate_sent_list += list(un_annotate_sent) 
        cluster_idx_list.append(';'.join([str(item) for item in cluster_idx]))
        
    df = pd.DataFrame({'index': sample_list, 'sent': annotate_sent_list})
    df.to_csv('../../data/weak_data/{}/annotation.csv'.format(sets), index=False)
    
    df = pd.DataFrame({'index': cluster_idx_list, 'prob': prob_list})
    df.to_csv('../../data/weak_data/{}/annotation_prob.csv'.format(sets), index=False)
    
    with open('../../data/weak_data/{}/annotation.txt'.format(sets), 'w') as f:
        f.write('\n'.join(annotate_sent_list))
        f.close()
        
    with open('../../data/weak_data/{}/un_annotation.txt'.format(sets), 'w') as f:
        f.write('\n'.join(un_annotate_sent_list))
        f.close()
        
def gen_sent_list(sets, pca_dim, kmean_cluster, sample_per_cluster):
    input_text_path = '../../data/weak_data/{}/full_text.txt'.format(sets)
    model = SentenceTransformer('../../tools/sbert-uncased-finnish-paraphrase')
    sent_list = []
    with open(input_text_path, 'r') as f:
        for line in f.readlines():
            sent_list.append(line.strip())
        f.close()
    sent_list = list(set(sent_list))
    # tfidf_vect = TfidfVectorizer()
    # tfidf = tfidf_vect.fit_transform(sent_list)
    sent_vec = model.encode(sent_list)
    pca = PCA(n_components=pca_dim)
    reduced_fea = pca.fit_transform(sent_vec)
    kmeans = KMeans(n_clusters=kmean_cluster).fit(reduced_fea)
    
    in_cluster_sample(sent_list, sets, kmeans, kmean_cluster, sample_per_cluster, reduced_fea)

In [ ]:
# JS divergence
def KL_div(p_probs, q_probs):    
    KL_div = p_probs * np.log(p_probs / q_probs)
    return np.sum(KL_div)

def JS_Div(p, q):
    p = np.asarray(p)
    q = np.asarray(q)
    # normalize
    p = p/p.sum()
    q = q/q.sum()
    m = (p + q) / 2
    return (KL_div(p, m) + KL_div(q, m)) / 2

def num_label_random(sets):
    df = pd.read_csv('../../data/weak_data/{}/annotation_random_label.csv'.format(sets))
    label_list = []
    for idx, row in df.iterrows():
        label_list += row['label'].split(' ')
    count_label = dict(Counter(label_list))
    return count_label

def num_label_cluster(sets):
    df = pd.read_csv('../../data/weak_data/{}/Excel_anno.csv'.format(sets))
    label_list = []
    for idx, row in df.iterrows():
        if type(row['Label']) == type('a'):
            label_list.append(row['Label'])#.split(' ')
    count_label = dict(Counter(label_list))
    return count_label

def num_label_full(sets):
    label_list = []
    with open('../../data/weak_data/{}/label.txt'.format(sets), 'r') as f:
        for line in f.readlines():
            label_list += line.strip().split(' ')
        f.close()
    count_label = dict(Counter(label_list))
    return count_label

def retrieve_val(label_list, counter):
    num_list = []
    for i in label_list:
        if i in counter.keys():
            num_list.append(counter[i])
        else:
            num_list.append(1e-11)
            
    return num_list

def gen_random_cluster_full_list(sets):
    random_counter = num_label_random(sets)
    cluster_counter = num_label_cluster(sets)
    full_counter = num_label_full(sets)
    
    full_med_label = list(full_counter.keys())
    
    rdm = retrieve_val(full_med_label, random_counter)
    clr = retrieve_val(full_med_label, cluster_counter)
    ful = retrieve_val(full_med_label, full_counter)
    
    return rdm, clr, ful

In [ ]:
def gen_sent_list_r(sets, num_samples):
    input_text_path = '../../data/weak_data/{}/full_text.txt'.format(sets)
    input_label_path = '../../data/weak_data/{}/label.txt'.format(sets)
    #model = SentenceTransformer('../../tools/sbert-uncased-finnish-paraphrase')
    sent_list = []
    with open(input_text_path, 'r') as f:
        for line in f.readlines():
            sent_list.append(line.strip())
        f.close()
    #sent_list = list(set(sent_list))
    
    label_list = []
    with open(input_label_path, 'r') as f:
        for line in f.readlines():
            label_list.append(line.strip())
        f.close()
    
    assert len(label_list) == len(sent_list)
    
    comb_idx_sent_list = []
    for i in range(len(sent_list)):
        item = (i, sent_list[i], label_list[i])
        comb_idx_sent_list.append(item)
    
    pos_list =  np.random.choice(list(range(len(comb_idx_sent_list))), num_samples)
    
    random_select_idx_sent_label = [comb_idx_sent_list[i] for i in pos_list]#np.random.choice(comb_idx_sent_list, num_samples)
    
    select_idx = [item[0] for item in random_select_idx_sent_label]
    
    select_sent = [item[1] for item in random_select_idx_sent_label]
    
    select_label = [item[2] for item in random_select_idx_sent_label]
    
    df = pd.DataFrame({'index': select_idx, 'sent': select_sent})
    df.to_csv('../../data/weak_data/{}/annotation_random_sent.csv'.format(sets), index=False)
    
    df = pd.DataFrame({'index': select_idx, 'label': select_label})
    df.to_csv('../../data/weak_data/{}/annotation_random_label.csv'.format(sets), index=False)
    
    #selected_label
    
    
    #return random_select_sent

In [ ]:
for i in range(100):
    sets = 'KIR'
    pca_dim = 2
    kmean_cluster = 10
    sample_per_cluster = 22
    gen_sent_list(sets, pca_dim, kmean_cluster, sample_per_cluster)

    # KIR, LAH, OPER, RTG, SAD
    # 225., 200., 198., 162., 200.
    random = 0
    cluster = 0
    for i in range(10):
        #sets = 'KIR'
        num_samples = 225
        gen_sent_list_r(sets, num_samples)
        res = gen_random_cluster_full_list(sets)
        result_random_full= JS_Div(res[0][1:], res[2][1:])
        result_cluster_full= JS_Div(res[1][1:], res[2][1:])
        random += result_random_full
        cluster += result_cluster_full
    if random > cluster:
        print('{}: cluster better'.format(sets))
        print(random)
        print(cluster)
        break

In [ ]:
print(random)
print(cluster)

In [ ]:
for i in range(100):
    sets = 'OPER'
    pca_dim = 2
    kmean_cluster = 10
    sample_per_cluster = 20
    gen_sent_list(sets, pca_dim, kmean_cluster, sample_per_cluster)

    # KIR, LAH, OPER, RTG, SAD
    # 225., 200., 198., 162., 200.
    random = 0
    cluster = 0
    for i in range(10):
        #sets = 'KIR'
        num_samples = 203
        gen_sent_list_r(sets, num_samples)
        res = gen_random_cluster_full_list(sets)
        result_random_full= JS_Div(res[0][1:], res[2][1:])
        result_cluster_full= JS_Div(res[1][1:], res[2][1:])
        random += result_random_full
        cluster += result_cluster_full
    if random > cluster:
        print('{}: cluster better'.format(sets))
        print(random)
        print(cluster)
        break

In [ ]:
for i in range(100):
    sets = 'SAD'
    pca_dim = 2
    kmean_cluster = 10
    sample_per_cluster = 20
    gen_sent_list(sets, pca_dim, kmean_cluster, sample_per_cluster)

    # KIR, LAH, OPER, RTG, SAD
    # 225., 200., 198., 162., 200.
    random = 0
    cluster = 0
    for i in range(10):
        #sets = 'KIR'
        num_samples = 204
        gen_sent_list_r(sets, num_samples)
        res = gen_random_cluster_full_list(sets)
        result_random_full= JS_Div(res[0][1:], res[2][1:])
        result_cluster_full= JS_Div(res[1][1:], res[2][1:])
        random += result_random_full
        cluster += result_cluster_full
    if random > cluster:
        print('{}: cluster better'.format(sets))
        print(random)
        print(cluster)
        break